In [3]:
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np


from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data/', one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [5]:
# 训练参数
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 500

In [6]:
# 神经网络参数
num_input = 28
timesteps = 28
num_hidden = 128
num_classes = 10

In [7]:
# tf 图形输入
X = tf.placeholder('float', [None, timesteps, num_input])
Y = tf.placeholder('float', [None, num_classes])

In [73]:
# 定义权重
weights = {
    'out':tf.Variable(tf.random_normal([2 * num_hidden, num_classes]))
}
biases = {
    'out':tf.Variable(tf.random_normal([num_classes]))
}

In [74]:
def BiRNN(x, weight, biases):
    with tf.variable_scope('BiRNN', reuse=tf.AUTO_REUSE):
        x = tf.unstack(x, timesteps, 1)
        lstm_fw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
        lstm_bw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
        try:
            outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell, 
                                                     lstm_bw_cell, x, dtype=tf.float32)
        except Exception:
            outputs = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x, dtype=tf.float32)
    return tf.matmul(outputs[-1], weight['out']) + biases['out']

In [76]:
logits = BiRNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [78]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(tf.global_variables_initializer())

    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step 1, Minibatch Loss= 2.7886, Training Accuracy= 0.141
Step 500, Minibatch Loss= 1.9442, Training Accuracy= 0.352
Step 1000, Minibatch Loss= 1.7257, Training Accuracy= 0.453
Step 1500, Minibatch Loss= 1.3891, Training Accuracy= 0.547
Step 2000, Minibatch Loss= 1.1426, Training Accuracy= 0.688
Step 2500, Minibatch Loss= 1.1836, Training Accuracy= 0.617
Step 3000, Minibatch Loss= 1.1274, Training Accuracy= 0.648
Step 3500, Minibatch Loss= 1.0606, Training Accuracy= 0.680
Step 4000, Minibatch Loss= 0.9879, Training Accuracy= 0.656
Step 4500, Minibatch Loss= 0.8396, Training Accuracy= 0.703
Step 5000, Minibatch Loss= 0.7576, Training Accuracy= 0.812
Step 5500, Minibatch Loss= 0.7277, Training Accuracy= 0.797
Step 6000, Minibatch Loss= 0.7012, Training Accuracy= 0.789
Step 6500, Minibatch Loss= 0.8003, Training Accuracy= 0.766
Step 7000, Minibatch Loss= 0.8509, Training Accuracy= 0.734
Step 7500, Minibatch Loss= 0.6206, Training Accuracy= 0.812
Step 8000, Minibatch Loss= 0.5515, Training 